# Cayley-Dickson Construction Applied to Qi Definition

*Version 1*

My original implementation of Gaussian integers included two classes, ``Zi`` and ``Qi``, where, for example, ``Zi(2, -7)`` represents a Gaussian integer, and ``Qi(-2/3, 4/5)`` represents a Gaussian rational.

I'd like to extend this code to include rational-valued quaternions and octonions using the Cayley-Dickson construction.

The [Cayley-Dickson construction](https://en.wikipedia.org/wiki/Cayley%E2%80%93Dickson_construction) is a process by which one can use a recursive definition of conjugation together with a recursive definition of multiplication to use...
* pairs of real numbers ($\mathbb{R}$) to create complex numbers,
* pairs of complex numbers ($\mathbb{C}$) to create quaternions,
* pairs of quaternions ($\mathbb{H}$) to create octonions,
* pairs of octonions ($\mathbb{O}$) to create sedenions ($\mathbb{S}$), and so on.

For more specifics, see my write-up about the Cayley-Dickson construction [at this link](https://abstract-algebra.readthedocs.io/en/latest/55_cayley_dickson.html).

In [1]:
from cayley_dickson_alg import Zi, SetScalarMult
from random import randint

In [2]:
from random import seed

seed(42)  # Generate the same random sequence each time (for testing)

In [3]:
from fractions import Fraction
from functools import wraps

In [4]:
def gaussian_rational(fnc):
    """For use as a property that casts an argument into Gaussian rational."""
    @wraps(fnc)
    def gaussian_rational_wrapper(arg, num):
        qi = to_gaussian_rational(num)
        return fnc(arg, qi)
    return gaussian_rational_wrapper

In [28]:
class Qi():
    """Gaussian Rational Number Class"""

    __max_denominator = 1_000_000

    def __init__(self, re=None, im=None):

        # --------------------------------------------------------
        # re is a str, int, float, or Fraction and
        # im is a str, int, float, Fraction or None

        if isinstance(re, Fraction):
            self.__re = re
            
        elif isinstance(re, (str, int, float)):
            self.__re = Fraction(re).limit_denominator(self.__max_denominator)
            if im is None:
                self.__im = Fraction(0)
            elif isinstance(im, (str, int, float)):
                self.__im = Fraction(im).limit_denominator(self.__max_denominator)
            else:
                raise Exception(f"Inputs incompatible: {re} and {im}")

    @classmethod
    def max_denominator(cls):
        return cls.__max_denominator

    @classmethod
    def set_max_denominator(cls, value):
        if value > 1:
            cls.__max_denominator = value
            return cls.__max_denominator
        else:
            raise ValueError("max_denominator must be > 1")

    @property
    def real(self) -> Fraction:
        return self.__re

    @property
    def imag(self) -> Fraction:
        return self.__im

    def __repr__(self):
        return f"Qi({repr(str(self.__re))}, {repr(str(self.__im))})"

    def __str__(self):
        if self.__im < 0:
            return f"({self.__re}{self.__im}j)"
        else:
            return f"({self.__re}+{self.__im}j)"

    @gaussian_rational
    def __add__(self, other):
        return Qi(self.__re + other.real, self.__im + other.imag)

In [ ]:
        # --------------------------------------------------------
        # re is a complex, and im is None, a complex, or a Zi

        elif isinstance(re, complex):
            if im is None:  #
                self.__re = round(re.real)
                self.__im = round(re.imag)
            elif isinstance(im, (complex, Zi)):
                self.__re = Zi(re)
                self.__im = Zi(im)
            else:
                raise Exception(f"Inputs incompatible: {re} and {im}")

        # --------------------------------------------------------
        # re is a Zi, and im is None, a complex, or a Zi

        elif isinstance(re, Zi):
            if im is None:
                self.__re = re.real
                self.__im = re.imag
            elif isinstance(im, (complex, Zi)):
                self.__re = Zi(re)
                self.__im = Zi(im)
            else:
                raise Exception(f"Inputs incompatible: {re} and {im}")

        # --------------------------------------------------------
        # re is a list or tuple of numbers with length equal to a
        # power of 2, and im is None, or it is a tuple or list
        # similar to the one input for re.
        elif isinstance(re, (tuple, list)):
            z = Zi.from_array(re)
            if im is None:
                self.__re = z.real
                self.__im = z.imag
            elif isinstance(im, (tuple, list)) and len(im) == len(re):
                w = Zi.from_array(im)
                self.__re = z
                self.__im = w
            else:
                raise Exception(f"Inputs incompatible: {re} and {im}")

        # --------------------------------------------------------
        # Both re and im are None

        elif re is None:
            self.__re = 0
            if im is None:
                self.__im = 0
            else:
                raise Exception(f"If re is None, then im must be None. But im = {im}")
        else:
            raise Exception(f"Unexpected combination of input types: {re} and {im}")
    

In [29]:
half = Fraction(1, 2)
print(half)

1/2


In [30]:
print(Qi('1/2', '2/3'))

(1/2+2/3j)


In [27]:
Qi((1-2j))

Qi('1', '0')